<a href="https://colab.research.google.com/github/harshraj7795/Parameter-Optimizaiton-CNN/blob/main/Image_CNN_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=4d79cd6a62b2d264736fba15f7149e3d10a804cce8c0e03fefa46854f1c31958
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=f7a690e9b33dfb47a8d5f0731b385063ae44a6a90ac9f0d01bfdae49965f50cb
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [ ]:
fm=keras.datasets.fashion_mnist

In [ ]:
(train_img,train_lab),(test_img,test_lab)=fm.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
#rescaling the image pixel values between 0 and 1
train_img=train_img/255.0
test_img=test_img/255.0

In [ ]:
train_img[0].shape

(28, 28)

In [ ]:
#reshaping the datasets
train_img=train_img.reshape(len(train_img),28,28,1)
test_img=test_img.reshape(len(test_img),28,28,1)

In [ ]:
#function for keras tuner
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
#importing libraries for tuning the model
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
#tuning the hyperparameters
tuner_search.search(train_img,train_lab,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 08m 04s]
val_accuracy: 0.9070000052452087

Best val_accuracy So Far: 0.9120000004768372
Total elapsed time: 01h 09m 58s
INFO:tensorflow:Oracle triggered exit


In [ ]:
#determining the best model
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
#print the summary of the best model
#The raw image data goes through these subsequent layers
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        38432     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               1734768   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 1,774,810
Trainable params: 1,774,810
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(train_img, train_lab, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1272 - accuracy: 0.9537 - val_loss: 0.2529 - val_accuracy: 0.9145
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0890 - accuracy: 0.9668 - val_loss: 0.2691 - val_accuracy: 0.9152
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0604 - accuracy: 0.9773 - val_loss: 0.3409 - val_accuracy: 0.9113
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0457 - accuracy: 0.9832 - val_loss: 0.3495 - val_accuracy: 0.9103
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0334 - accuracy: 0.9880 - val_loss: 0.4110 - val_accuracy: 0.9137
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0248 - accuracy: 0.9912 - val_loss: 0.5651 - val_accuracy: 0.9122
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0214 - accuracy: 0.9925 - val_loss: 0.5133 - val_accuracy